In [ ]:
# get data using curl
!curl https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/e9a18c27-67da-40a2-8f48-1c12f03de39d -o data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8050M  100 8050M    0     0  20.7M      0  0:06:27  0:06:27 --:--:-- 23.6M


In [ ]:
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: CellData/OCT/train/DME/DME-778975-125.jpeg  
  inflating: CellData/OCT/train/DME/DME-258763-26.jpeg  
  inflating: CellData/OCT/train/DME/DME-8457431-23.jpeg  
  inflating: CellData/OCT/train/DME/DME-3304080-55.jpeg  
  inflating: CellData/OCT/train/DME/DME-5006393-70.jpeg  
  inflating: CellData/OCT/train/DME/DME-5091309-15.jpeg  
  inflating: CellData/OCT/train/DME/DME-3064922-181.jpeg  
  inflating: CellData/OCT/train/DME/DME-8525575-184.jpeg  
  inflating: CellData/OCT/train/DME/DME-5143281-26.jpeg  
  inflating: CellData/OCT/train/DME/DME-6737988-35.jpeg  
  inflating: CellData/OCT/train/DME/DME-3064922-202.jpeg  
  inflating: CellData/OCT/train/DME/DME-3531101-20.jpeg  
  inflating: CellData/OCT/train/DME/DME-3157783-58.jpeg  
  inflating: CellData/OCT/train/DME/DME-258763-41.jpeg  
  inflating: CellData/OCT/train/DME/DME-30521-85.jpeg  
  inflating: CellData/OCT/train/DME/DME-5489583-15.jpeg  
  inflating: CellData/

install ML Flow and Pyngrok

In [ ]:
!pip install mlflow pyngrok

In [ ]:
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok auth token
NGROK_AUTH_TOKEN = "2px4g5pyfnotmAh0CZCYBJgn5mE_2oVQnsqYaUjnpvdvmoL6H"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Start the MLflow UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:
# Kill any existing ngrok tunnels
ngrok.kill()

# Open a new HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://4859-34-34-106-78.ngrok-free.app


In [ ]:
!pip install torchmetrics


## Imports

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time
import copy
import tqdm.notebook as tqdm

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as T
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

#Image packages
from PIL import Image
import mlflow
import torchmetrics


In [ ]:
# Set experiment name
#get date
from datetime import datetime
now = datetime.now()
# Format the date and time as a string (e.g., "2024-12-09_12-00-00")
experiment_name = now.strftime("%Y-%m-%d_%H-%M-%S")
mlflow.set_experiment("" + experiment_name)
mlflow.pytorch.autolog()

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


## Load Data

In [ ]:
# mydir = os.getcwd()
mydir = '/content/project/'
train_dir = os.path.join(mydir, 'chest_xray', 'train')
test_dir = os.path.join(mydir, 'chest_xray', 'test')

In [ ]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

cnn_transforms =  T.Compose([
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=mean_nums, std=std_nums),
                    ])

# data_transforms = {"train": transforms.Compose([
#                                             transforms.Resize(256),
#                                             transforms.CenterCrop(224),
#                                             transforms.ToTensor(),
#                                             transforms.Normalize(mean=mean_nums, std=std_nums),
#                                         ]),
#                     "test": transforms.Compose([
#                                 transforms.Resize(256),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor(),
#                                 transforms.Normalize(mean=mean_nums, std=std_nums),
#                             ]),}

In [ ]:
def create_split_train_val_test(train_dir, test_dir, transforms, val_size=0.15):
    train_data = datasets.ImageFolder(train_dir,
                    transform=transforms)
    test_data = datasets.ImageFolder(test_dir,
                    transform=transforms)
    train_subset, val_subset = torch.utils.data.random_split(train_data, [1-val_size, val_size], generator=torch.Generator().manual_seed(1))
    torch.save(train_subset, mydir+'/train_data.pt')
    torch.save(val_subset, mydir+'/val_data.pt')
    torch.save(test_data, mydir+'/test_data.pt')

def load_data(dir, batch_size=64):
    train_data = torch.load(os.path.join(dir, 'train_data.pt'))
    val_data = torch.load(os.path.join(dir, 'val_data.pt'))
    test_data = torch.load(os.path.join(dir, 'test_data.pt'))
    train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=3)
    val_loader = torch.utils.data.DataLoader(val_data, shuffle=True, batch_size=batch_size, num_workers=3)
    test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=batch_size, num_workers=3)
    return train_loader, val_loader, test_loader

In [ ]:
create_split_train_val_test(train_dir, test_dir, cnn_transforms)

In [ ]:
batch_size = 64
train_loader, val_loader, test_loader = load_data(mydir, batch_size)

<ipython-input-81-c70a5cbcdf4e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))
<ipython-input-81-c70a5cbcdf4e>

In [ ]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [ ]:
class_names = train_loader.dataset.dataset.classes

In [ ]:
def compute_metrics(y_true, y_pred):
    """
    Compute precision, recall, and F1 score using sklearn metrics.
    """
    # Initialize precision, recall, and F1 score trackers for multi-class classification
    precision_metric = torchmetrics.Precision(num_classes=2, average='macro',task='binary').to(device)  # adjust num_classes
    recall_metric = torchmetrics.Recall(num_classes=2, average='macro',task='binary').to(device)
    f1_metric = torchmetrics.F1Score(num_classes=2, average='macro',task='binary').to(device)

## Model Train Function

In [ ]:
def train_model(model, dataloaders, data_sizes, criterion, optimizer, scheduler, device, batch_size, num_epochs=10):
    # Start the MLflow run
    with mlflow.start_run():

        # Log model hyperparameters
        params = {
            "epochs": num_epochs,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "batch_size": batch_size,
            "loss_function": type(criterion).__name__,
            "scheduler": type(scheduler).__name__,
            "optimizer": type(optimizer).__name__,
            "model": type(model).__name__
        }
        mlflow.log_params(params)

        since = time.time()

        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = np.inf

        # Training Loop
        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluation mode

                current_loss = 0.0
                current_corrects = 0
                all_preds = []
                all_labels = []

                for inputs, labels in tqdm.tqdm(dataloaders[phase], desc=phase, leave=False):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    if phase == 'train':
                        scheduler.step()

                    current_loss += loss.item() * inputs.size(0)
                    current_corrects += torch.sum(preds == labels.data)

                    # Collect all predictions and labels for metrics calculation
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

                # Calculate loss and accuracy for this epoch
                epoch_loss = current_loss / data_sizes[phase]
                epoch_acc = current_corrects.double() / data_sizes[phase]

                # Calculate precision, recall, and F1 score
                precision, recall, f1 = compute_metrics(all_labels, all_preds)

                # Log metrics to MLflow
                mlflow.log_metric(f'{phase}_loss', epoch_loss, step=epoch)
                mlflow.log_metric(f'{phase}_accuracy', epoch_acc, step=epoch)
                mlflow.log_metric(f'{phase}_precision', precision, step=epoch)
                mlflow.log_metric(f'{phase}_recall', recall, step=epoch)
                mlflow.log_metric(f'{phase}_f1_score', f1, step=epoch)

                # Print epoch results
                print(f'{phase} Loss: {epoch_loss:.4f} | {phase} Accuracy: {epoch_acc:.4f}')
                print(f'{phase} Precision: {precision:.4f} | {phase} Recall: {recall:.4f} | {phase} F1 Score: {f1:.4f}')

                # Early stopping: Save the best model
                if phase == 'val' and epoch_loss < best_loss:
                    print(f'Val loss decreased from {best_loss:.4f} to {epoch_loss:.4f}. Saving weights...')
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())

            print()

        # Record the time spent training
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best validation loss: {best_loss:.4f}')

        # Log final metrics to MLflow
        mlflow.log_metric("best_val_loss", best_loss)
        mlflow.log_metric("training_time", time_elapsed)

        # Load the best model weights
        model.load_state_dict(best_model_wts)

        # Save the model to MLflow
        mlflow.pytorch.log_model(model, "model")

    return model

In [ ]:
'''
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    mlflow.start_run()
    params = {
        "epochs": num_epochs,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "batch_size": batch_size,
        "loss_function": type(criterion).__name__ ,
        "scheduler": type(scheduler).__name__,
        "optimizer": type(optimizer).__name__,
        "model": type(model).__name__
    }
    mlflow.log_params(params)

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10) # Log model hyperparameters

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            for inputs, labels in tqdm.tqdm(dataloaders[phase], desc=phase, leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients in the Cache.
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                if phase == 'train':
                    scheduler.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
            epoch_loss = current_loss / data_sizes[phase]
            epoch_acc = current_corrects.double() / data_sizes[phase]
            if phase == 'val':
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))
            else:
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))

            # EARLY STOPPING
            if phase == 'val' and epoch_loss < best_loss:
                print('Val loss Decreased from {:.4f} to {:.4f} \nSaving Weights... '.format(best_loss, epoch_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val loss: {:.4f}'.format(best_loss))
    mlflow.log_metric("best_val_loss", best_loss)
    mlflow.log_metric("training_time", time_since)

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")
    return model
'''

## Densenet Model

In [ ]:
def Densenet_Model(class_names, pretrained=True):
    if pretrained:
        model = models.densenet121(weights='DEFAULT')
    else:
        model = models.densenet121(weights=None)

    for params in model.parameters():
        params.requires_grad=False
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model



In [ ]:

# Define the training function for Ray Tune
def train_model(lr, gamma, step_size, train_loader, device,num_epochs):



    # Initialize model, loss function, optimizer
    dense_model = Densenet_Model(class_names, pretrained=True).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(dense_model.parameters(), lr=lr)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop (simplified)
    num_epochs = 10
    total_accuracy = 0

    for epoch in range(num_epochs):
        dense_model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Ensure data is on the same device as the model

            optimizer.zero_grad()
            outputs = dense_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        exp_lr_scheduler.step()

        # Calculate accuracy
        accuracy = 100 * correct / total
        total_accuracy += accuracy
        #tune.report(accuracy=accuracy)  # Report accuracy to Ray Tune
    return total_accuracy / num_epochs  # Return average accuracy


# Define the search space for hyperparameters

# Example Hyperparameter Search Space (Grid Search)
learning_rates = [1e-5, 1e-4, 1e-3]  # Learning rates to try
gammas = [0.1, 0.5, 0.9]  # Gamma values for learning rate decay
step_sizes = [5, 10, 15]  # Step sizes for LR scheduler

# Initialize training configurations
best_accuracy = 0
best_params = {}

# Assume `train_loader` is predefined (data loading should be done earlier)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Grid Search over all combinations of hyperparameters
for lr in learning_rates:
    for gamma in gammas:
        for step_size in step_sizes:
            print(f"Training with lr={lr}, gamma={gamma}, step_size={step_size}")

            # Train the model with current hyperparameters
            avg_accuracy = train_model(lr, gamma, step_size, train_loader, device)
            print(f"Avg Accuracy: {avg_accuracy}")

            # Update the best parameters if the current run is better
            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'lr': lr, 'gamma': gamma, 'step_size': step_size}

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}")

# Get the best hyperparameters
#print("Best Hyperparameters: ", analysis.best_config)

ModuleNotFoundError: No module named 'ray'

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [ ]:
pytorch_total_params = sum(p.numel() for p in dense_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
2050


In [ ]:
best_densenet_model = train_model(dense_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

TypeError: train_model() got an unexpected keyword argument 'num_epochs'

In [ ]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_densenet_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(test_loss)
print(test_acc)

0.691200350339596
tensor(0.6250, device='cuda:0', dtype=torch.float64)


In [ ]:
torch.save(best_densenet_model.state_dict(), 'best_densenet_model.pth')

In [ ]:
mlflow.end_run()

## Resnet Model

In [ ]:
def Resnet_Model(class_names, pretrained=True):
    if pretrained:
        model = models.resnet50(weights='DEFAULT')
    else:
        model = models.resnet50(weights=None)
    for params in model.parameters():
        params.requires_grad=False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model

res_model = Resnet_Model(class_names, pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss()

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(res_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [ ]:
pytorch_total_params = sum(p.numel() for p in res_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
4098


In [ ]:
best_resnet_model = train_model(res_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Exception: Run with UUID 79d74391e3e949c9ab0a4d4c5277840a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_resnet_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(test_loss)
print(test_acc)

0.635118889885071
tensor(0.6250, device='cuda:0', dtype=torch.float64)


In [ ]:
torch.save(best_resnet_model.state_dict(), 'best_resnet_model.pth')

## Transformer

In [ ]:
def Transformer_Model(class_names, pretrained=True):
    if pretrained:
        model = models.vit_b_16(weights='DEFAULT')
    else:
        model = models.resnet50(weights=None)

    weights = models.ViT_B_16_Weights.DEFAULT
    transforms = weights.transforms()
    #Freeze transformer
    for params in model.parameters():
        params.requires_grad=False

    num_ftrs = model.hidden_dim
    model.heads = nn.Linear(num_ftrs, len(class_names))
    model = model.to(device)
    return model, transforms

transformer_model, transformer_transforms = Transformer_Model(class_names, pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss()

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(transformer_model.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [ ]:
transformer_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
pytorch_total_params = sum(p.numel() for p in transformer_model.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
1538


In [ ]:
create_split_train_val_test(train_dir, test_dir, transformer_transforms)

In [ ]:
train_loader, val_loader, test_loader = load_data(mydir, 64)

C:\Users\rohil\AppData\Local\Temp\ipykernel_13828\3774757452.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))

In [ ]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [ ]:
best_transformer_model = train_model(transformer_model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

c:\Users\rohil\anaconda3\envs\cs7643-a3\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


train Loss: 0.3177 | train Accuracy: 0.8934


val:   0%|          | 0/13 [00:00<?, ?it/s]

...

val Loss: 0.2980 | val Accuracy: 0.9094
Val loss Decreased from 0.2980 to 0.2980 
Saving Weights... 

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.3019 | train Accuracy: 0.9080


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2980 | val Accuracy: 0.9094

Training complete in 23m 60s
Best val loss: 0.2980


In [ ]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_transformer_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(test_loss)
print(test_acc)

0.4429000127009856
tensor(0.7804, device='cuda:0', dtype=torch.float64)


In [ ]:
torch.save(best_transformer_model.state_dict(), 'best_transformer_model.pth')

## Hybrid Resnet + Transformer

In [ ]:
class hybrid_model(nn.Module):
    def __init__(self, vit_model, resnet_model, dropout=0.3, device=device):
        super().__init__()
        self.vit_model = vit_model.to(device)
        self.vit_linear = nn.Linear(768, 2048, device=device)

        self.resnet_model = resnet_model.to(device)

        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(4096, 500, device=device)
        self.linear2 = nn.Linear(500, 2, device=device)

    def forward(self, inputs):
        vit_result = self.vit_model(inputs)['getitem_5']
        vit_result = self.vit_linear(vit_result)

        res_result = self.resnet_model(inputs)
        res_result = torch.reshape(res_result, (res_result.shape[0], res_result.shape[1]))

        output = torch.cat((vit_result, res_result), 1)
        output = self.dropout(output)
        output = self.linear1(output)
        output = self.dropout(output)
        output = self.linear2(output)
        return output


In [ ]:
def hybrid(class_names):
    resnet_model = models.resnet50(weights='DEFAULT')
    vit_model = models.vit_b_16(weights='DEFAULT')

    # Remove the last layer of Resnet
    resnet_layers = list(resnet_model.children())
    resnet_model = nn.Sequential(*resnet_layers[:-1])

    for params in resnet_model.parameters():
        params.requires_grad=False


    for params in vit_model.parameters():
        params.requires_grad=False

    vit_model_top = create_feature_extractor(vit_model, return_nodes=['getitem_5'])
    model = hybrid_model(vit_model_top, resnet_model, 0.3, device)

    model = model.to(device)
    return model

hmodel = hybrid(class_names)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss()

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(hmodel.parameters(), lr=1e-3)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [ ]:
full_train_df = {}
labels = ['PNEUMONIA', 'NORMAL']
i = 0
for label in labels:
    path = os.path.join(train_dir, label)
    for img in os.listdir(path):
        full_train_df[i] = [os.path.join(path, img), label]
        i += 1

full_train_df = pd.DataFrame.from_dict(data=full_train_df, orient='index', columns=['file_path','labels'])
train_df = full_train_df.sample(frac=0.85, random_state=42).sample(frac=1.0, random_state=42)
val_df = full_train_df.drop(train_df.index).sample(frac=1.0, random_state=42)


test_df = {}
i = 0
for label in labels:
    path = os.path.join(test_dir, label)
    for img in os.listdir(path):
        test_df[i] = [os.path.join(path, img), label]
        i += 1

test_df = pd.DataFrame.from_dict(data=test_df, orient='index', columns=['file_path','labels'])

In [ ]:
create_split_train_val_test(train_dir, test_dir, transformer_transforms)

In [ ]:
train_loader, val_loader, test_loader = load_data(mydir, 64)

C:\Users\rohil\AppData\Local\Temp\ipykernel_13828\3774757452.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(os.path.join(dir, 'train_data.pt'))

In [ ]:
dataloaders = {"train":train_loader, "val":val_loader}
data_sizes = {x: len(dataloaders[x].sampler) for x in ['train','val']}

In [ ]:
pytorch_total_params = sum(p.numel() for p in hmodel.parameters() if p.requires_grad)
print("Number of trainable parameters: \n{}".format(pytorch_total_params))

Number of trainable parameters: 
3624414


In [ ]:
def train_hybrid_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            for inputs, labels in tqdm.tqdm(dataloaders[phase], desc=phase, leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients in the Cache.
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                if phase == 'train':
                    scheduler.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
            epoch_loss = current_loss / data_sizes[phase]
            epoch_acc = current_corrects.double() / data_sizes[phase]
            if phase == 'val':
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))
            else:
                print('{} Loss: {:.4f} | {} Accuracy: {:.4f}'.format(
                    phase, epoch_loss, phase, epoch_acc))

            # EARLY STOPPING
            if phase == 'val' and epoch_loss < best_loss:
                print('Val loss Decreased from {:.4f} to {:.4f} \nSaving Weights... '.format(best_loss, epoch_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val loss: {:.4f}'.format(best_loss))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
best_hybrid_model = train_hybrid_model(hmodel, criterion, optimizer, exp_lr_scheduler)

Epoch 1/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.3748 | train Accuracy: 0.9049


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2769 | val Accuracy: 0.9260
Val loss Decreased from inf to 0.2769 
Saving Weights... 

Epoch 2/10
----------


...

val Loss: 0.2782 | val Accuracy: 0.9286

Epoch 10/10
----------


train:   0%|          | 0/70 [00:00<?, ?it/s]

train Loss: 0.2713 | train Accuracy: 0.9254


val:   0%|          | 0/13 [00:00<?, ?it/s]

val Loss: 0.2790 | val Accuracy: 0.9273

Training complete in 26m 21s
Best val loss: 0.2769


In [ ]:
y_pred_list = []
y_true_list = []
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(test_loader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = best_hybrid_model(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        loss = criterion(y_test_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_corrects += torch.sum(y_pred_tag == y_batch.data)

test_loss = test_loss / len(test_loader.sampler)
test_acc = test_corrects.double() / len(test_loader.sampler)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(test_loss)
print(test_acc)

0.803626088377757
tensor(0.8333, device='cuda:0', dtype=torch.float64)


In [ ]:
torch.save(best_hybrid_model.state_dict(), 'best_hybrid_model.pth')